### _Extract Data_

In [19]:
import requests
import pandas as pd
import firebase_admin
from firebase_admin import storage
from firebase_admin import credentials

In [20]:
def extract_data() -> list:
    url = "https://gist.githubusercontent.com/nadirbslmh/93b62fdcfa694d4ec07bed9b3c94e401/raw/c07971341361e23fd4f3a880437c4d8e4ebcfafc/stock_trades.json"
    response = requests.get(url)
    if response.status_code == 200 :
        data = response.json()
        return data
    else :
        print("Failed to fetch data from the API.")
        return []

### _Transform Data_

In [24]:
def transform_data() -> pd.DataFrame:
    data = extract_data()
    df = pd.json_normalize(data)
    filtered_df = df[(df['stock_symbol'].isin(['GOOGL', 'AMZN', 'MSFT', 'AAPL'])) & (df['trade_price'] > 500)]
    filtered_df.reset_index(drop=True, inplace=True)
    return filtered_df

transform_data()

,trade_id,stock_symbol,trade_date,trade_time,trade_price,trade_volume,buyer_id,seller_id,trade_type,currency
0,1,GOOGL,1/5/2017,3,675.51,501030,1,1,sell,EUR
1,8,AMZN,11/27/2017,16,827.92,384783,8,8,buy,CNY
2,16,MSFT,1/1/2020,5,829.56,131961,16,16,sell,CUP
3,19,MSFT,11/9/2015,21,753.17,994135,19,19,sell,CNY
4,35,MSFT,7/12/2020,1,617.64,735687,35,35,buy,RUB
...,...,...,...,...,...,...,...,...,...,...
180,978,MSFT,7/7/2016,3,707.83,492637,978,978,sell,XAF
181,985,GOOGL,10/10/2013,16,923.32,246221,985,985,buy,SEK
182,989,AMZN,10/23/2019,7,736.30,941406,989,989,sell,BRL
183,995,MSFT,1/18/2014,7,763.06,332642,995,995,sell,IDR


### _Load Data_

In [23]:
def load_data() :
    try:
        data = transform_data()
        cred = credentials.Certificate('serviceAccountKey.json')
        storage_bucket = "project-1-418609.appspot.com"
        firebase_admin.initialize_app(cred, {"storageBucket": storage_bucket})
        bucket = storage.bucket()
        
        file_path = "data_source/"
        file_name = "stock_trades.parquet"
        data.to_parquet(file_path + file_name)
                
        blob = bucket.blob(file_name)
        blob.upload_from_filename(file_path + file_name)
        print(f"Data has been successfully loaded to the storage")
    except Exception as e:
        print(f"An error occurred when loading data to the storage: {e}")

load_data()

Data has been successfully loaded to the storage
